## Data Cleaning

In [3]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [4]:
df = pd.read_csv("../data/raw/phpMYEkMl.csv")
print(df.shape)
df.sample(3)

(1309, 14)


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1168,3,0,"Sadowitz, Mr. Harry",male,?,0,0,LP 1588,7.575,?,S,?,?,?
537,2,0,"Ponesell, Mr. Martin",male,34,0,0,250647,13,?,S,?,?,"Denmark / New York, NY"
498,2,0,"McCrae, Mr. Arthur Gordon",male,32,0,0,237216,13.5,?,S,?,209,"Sydney, Australia"


In [5]:
df = df.replace("?", np.nan)

In [ ]:
pr = ProfileReport(df)
pr.to_file(output_file="titanic_raw.html")

In [6]:
drop_cols = ["name", # high cardinality, uniform [99.8% distinct]
             "sibsp", # zeros [68.1% zeros]
             "parch", # zeros [76.5% zeros]
             "ticket", # high cardinality, uniform [71.0% distinct]
             "cabin", # missing [77.5% nulls]
             "boat", # missing [62.9% nulls]
             "body", # missing [90.8% nulls]
             "home.dest", # missing [43.1% nulls]
            ]

In [7]:
df = df.drop(columns=drop_cols)
df

,pclass,survived,sex,age,fare,embarked
0,1,1,female,29,211.3375,S
1,1,1,male,0.9167,151.55,S
2,1,0,female,2,151.55,S
3,1,0,male,30,151.55,S
4,1,0,female,25,151.55,S
...,...,...,...,...,...,...
1304,3,0,female,14.5,14.4542,C
1305,3,0,female,NaN,14.4542,C
1306,3,0,male,26.5,7.225,C
1307,3,0,male,27,7.225,C


In [8]:
df.isna().sum()

pclass        0
survived      0
sex           0
age         263
fare          1
embarked      2
dtype: int64

In [9]:
df.dtypes

pclass       int64
survived     int64
sex         object
age         object
fare        object
embarked    object
dtype: object

In [10]:
df["sex"] = np.where(df["sex"]=="female", 0, 1)
df["sex"] = df["sex"].astype("int64")

In [11]:
df["age"] = df["age"].astype("float")
df["age"] = df["age"].fillna(value=df["age"].mean())
df["age"] = round(df["age"],0)
df["age"] = df["age"].astype("int64")

In [12]:
df["fare"] = df["fare"].astype("float")
df["fare"] = df["fare"].fillna(value=df["fare"].mean())
df["fare"] = round(df["fare"],2)

In [13]:
df["embarked"] = df["embarked"].fillna(method='ffill').fillna(method='bfill')
df["embarked"] = df["embarked"].replace("S", 1)
df["embarked"] = df["embarked"].replace("C", 2)
df["embarked"] = df["embarked"].replace("Q", 3)
df["embarked"] = df["embarked"].astype("int64")

In [14]:
df.isna().sum()

pclass      0
survived    0
sex         0
age         0
fare        0
embarked    0
dtype: int64

In [15]:
df.dtypes

pclass        int64
survived      int64
sex           int64
age           int64
fare        float64
embarked      int64
dtype: object

In [16]:
df.to_csv("../data/clean/titanic.csv", index=False)
df

,pclass,survived,sex,age,fare,embarked
0,1,1,0,29,211.34,1
1,1,1,1,1,151.55,1
2,1,0,0,2,151.55,1
3,1,0,1,30,151.55,1
4,1,0,0,25,151.55,1
...,...,...,...,...,...,...
1304,3,0,0,14,14.45,2
1305,3,0,0,30,14.45,2
1306,3,0,1,26,7.22,2
1307,3,0,1,27,7.22,2


In [17]:
pr = ProfileReport(df)
pr.to_file(output_file="titanic_clean.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 391.37it/s]
